# 1- Agente Basico

Este código define una estructura para crear un esquema de agente base, con el cual se pueden crear multiples agentes como el caso de un agente de conversacional.

**Clase Agente (`Agent`)**: Esta es una clase abstracta base (`ABC`), que define una estructura general para un agente. Tiene un constructor que inicializa la memoria y el modelo LLM de OPENAI, y un método abstracto `run` que debe ser implementar el usuario que define el comportamiento del agente

## 1.1 - Importaciones y configuraciones

In [1]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
load_dotenv()

client = OpenAI()

## 1.2- Clase Agente

- `Agent` es una clase abstracta base (ABC) que define una estructura para otros agentes.
- Tiene un constructor que acepta un modelo de la API de OpenAI y un mensaje de sistema inicial.
- `add_to_memory` almacena los mensajes en memoria para futuras referencias.
- `get_schema` devuelve el esquema de entrada/salida del agente
- `run` es un método abstracto que debe ser implementado por las subclases para tomar una entrada del usuario y ejecutar el comportamiento del agente.

In [ ]:
class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""

## 1.3- Creando un Agente

La creación de un agente de conversación se realiza mediante la definición de una nueva clase como el caso de `ConversationAgent`, que hereda de la clase base abstracta `Agent`. Se implementa el metodo `run` para generar el algoritmo que el agente ejecutará.


In [2]:
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        agent_name: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.agent_name = agent_name
        self.description = description

        self.function={
            "name": self.agent_name,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        completion=''
        stream = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
            stream=True
        )
        print(f"{self.agent_name}:")
        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                #print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

    


## 1.4- Ejecutar agente

In [3]:
agent=ConversationAgent(
    model="gpt-4",
    agent_name="test-agent",
    description="A simple test agent",
    )

agent.run('Hello!')

test-agent:


'Hello! How can I assist you today?'